# PA 6 - Kevin Bacon Game
- Zach Fechko
- Last Updated: April 18, 2022
- Version 1.0
- Description: Uses an unweighted graph to implement the kevin bacon game, connecting actors to kevin bacon through movies they star in

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from rich.console import Console
import graphviz as gv
from collections import deque
import sys

## Functions that construct the dictionaries to create graphs

In [5]:
def create_movies_dict():
    """
    opens moviesTest.txt (will later be changed to movies.txt) and creates a dictionary by parsing each line of the file,
    returns a dictionary of movie ids and movie titles
    """
    movies = dict()
    with open('data/moviesTest.txt') as file:
        for line in file:
            k, v = line.strip().split('|')
            movies[k.strip()] = v.strip()
    return movies

def create_actors_dict():
    """
    opens actorsTest.txt (will later be changed to actors.txt) and creates a dictionary by parsing each line of the file,
    returns a dictionary of actor ids and actor names
    """
    actors = dict()
    with open('data/actorsTest.txt') as file:
        for line in file:
            k, v = line.strip().split('|')
            actors[k.strip()] = v.strip()
    return actors 


def create_pairs_dict():
    """
    opens movie-actorsTest.txt (will later be changed to movie-actors.txt) and parses the data, if the key doesn't exist, create a list at that index with the value
    if it does exist in the dict, just append the value to that list
    returns a dictionary made up movie ids, and a list of actor ids that star in that film
    """
    pairs = dict()
    with open('data/movie-actorsTest.txt') as file:
        for line in file:
            k, v = line.strip().split('|')
            if k not in pairs:
                pairs[k.strip()] = [v.strip()]
            else:
                pairs[k].append(v.strip())
    return pairs 

actors = create_actors_dict()
movies = create_movies_dict()
pairs = create_pairs_dict()

pairs
            

{'10': ['1', '100', '200'],
 '20': ['100', '300'],
 '30': ['300', '200'],
 '40': ['300', '400'],
 '50': ['500', '600']}

In [8]:
for movie in movies:
    print(movie)

10
20
30
40
50


Pseudo code
1. Movie-actors dictionary:
    - movie <-> [list of actors]
2. for each movie:
    - find the actors based on the ids in the list in the pairs dict
    - for actor 1 in list of actors
        - for actor 2 in list of actors
            - if actor1 != actor2:
                - add actor 1 to graph
                - add actor 2 to graph
                - add edge (movie) between them

In [33]:
class Vertex:
    def __init__(self, actor_ID, actor_name, predecessor=None, distance=0):
        self.actor_ID = actor_ID #this will take on the role of the key
        self.name = actor_name
        self.connected_to = dict()
        self.distance = distance
        self.predecessor = predecessor


    def add_neighbor(self, neighbor, movie_name: str):
        self.connected_to[neighbor] = movie_name

    def get_connections(self):
        return self.connected_to.keys()

    def get_ID(self):
        return self.actor_ID

    def get_name(self):
        return self.name

    def get_movie(self, neighbor): #get_weight
        return self.connected_to[neighbor]

    def get_distance(self):
        return self.distance 
    
    def get_predecessor(self):
        return self.predecessor 

    def set_distance(self, dist: int):
        self.distance = dist

    def set_predecessor(self, pred):
        self.predecessor = pred

    
class Graph:
    def __init__(self):
        self.vert_list = dict()
        self.movies = create_movies_dict()
        self.actors = create_actors_dict()
        self.pairs = create_pairs_dict()
        self.num_vertices = 0

    def add_vertex(self, id, distance=0, predecessor=None):
        """
        given an actor id from the pairs dictionary
        find actor name in actors dict
        create vertex object with actor name, and ID
        return the vertex
        """
        self.num_vertices += 1
        name = self.actors[id] # gives the name of the actor
        new_vertex = Vertex(id, name, predecessor, distance)
        self.vert_list[id] = new_vertex
        return new_vertex

    def __str__(self):
        '''

        '''
        edges = ""
        for vert in self.vert_list.values():
            for vert2 in vert.get_connections():
                edges += "(%s, %s: %s)\n" %(vert.get_name(), vert2.get_name(), vert.get_movie(vert2))
        return edges

    def add_edge(self, actor1, actor2, movie_id):
        """
        if actor1 and actor2 have a movie with the same id
        make an edge with the weight equal to that movie id
        """
        """
        if actor1.actor_id not in self.vert_list:
            nv = self.add_vertex(actor1)
        if actor2.actor_id not in self.vert_list:
            nv = self.add_vertex(actor2)
        """
        common_movie = self.movies[movie_id] #need to change this so it sets the edge weight to the name of the movie, not the id
         #if the two actors have an edge they can use
        self.vert_list[actor1.actor_ID].add_neighbor(self.vert_list[actor2.actor_ID], common_movie)

    def get_vertex(self, id):
        '''

        '''
        if id in self.vert_list:
            return self.vert_list[id]
        else:
            return None


    def build_graph(self):
        """
        return each vertex
        add edge between the two
        """
        for movie in self.movies:
            temp = []
            for actor in self.pairs[movie]:
                if actor not in self.vert_list: #if the actor id has not been added before
                    v = self.add_vertex(actor, sys.maxsize, None)
                else:
                    v = self.get_vertex(actor)
                temp.append(v)
            for v in temp:
                for i in range(len(temp)):
                    if temp[i] != v:
                        self.add_edge(v, temp[i], movie)
                        self.add_edge(temp[i], v, movie)
            



In [34]:
g = Graph()
g.build_graph()
print(g)

(Kevin Bacon, actor1: movie1)
(Kevin Bacon, actor2: movie1)
(actor1, Kevin Bacon: movie1)
(actor1, actor2: movie1)
(actor1, actor3: movie2)
(actor2, Kevin Bacon: movie1)
(actor2, actor1: movie1)
(actor2, actor3: movie3)
(actor3, actor1: movie2)
(actor3, actor2: movie3)
(actor3, actor4: movie4)
(actor4, actor3: movie4)
(actor5, actor6: movie5)
(actor6, actor5: movie5)



## Main game flow 
- create a vertex for every actor in actors.txt
- add edges between all the vertices
- prompt the user for an actor